# NBME - Problem Statement

**If you like the notebook do give an upvote for support :) Thank You.**

**医師の診察を受ける際、症状をどのように解釈するかで、診断が正確かどうかが決まります。医師は免許を取得するまでに、患者さんの訴えの経緯、身体検査所見、考えられる診断、経過観察などを記録する「患者メモ」を書く練習をたくさんしてきました。その際、他の医師からのフィードバックが必要ですが、機械学習により、このプロセスを改善することができます。このコンペティションの目標は、各患者メモの中から関連する特徴を自動的に特定する方法を開発することです。特に、標準的な患者との面接から得た情報が記録される患者履歴の部分に重点を置いています**。

**このコンペティションでは、患者メモに含まれる特定の臨床概念を特定します。具体的には、試験のルーブリックにある臨床概念（例：「食欲不振」）と、医学生が書いた臨床患者のメモにある様々な表現方法（例：「食事量が少ない」「服がゆったり着られる」）を自動的に対応付ける方法を開発していただきます**。

# ライブラリのインポート

In [ ]:
!pip install stylecloud

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
import string

import plotly.express as px
import plotly.graph_objects as go


import stylecloud
from IPython.display import Image

from warnings import filterwarnings
filterwarnings('ignore')

# データセットについて

**train.csv - 1000件の患者メモに対する特徴量注釈（10件ごとに100件）** **train.csv - 1000件の患者メモに対する特徴量注釈（10件ごとに100件）。

* **id** - 各患者ノートと特徴のペアの一意な識別子。
* **pn_num** - この行でアノテーションされた患者メモ。
* **feature_num** - この行で注釈された特徴量です。
* **case_num** - この患者メモが属するケースです。
* **annotation** - 患者ノート内の特徴を示すテキスト。注釈** - 患者ノート内の、特徴を示すテキスト。
* **location** - メモ内の各注釈の位置を示す文字列です。注釈を表現するために複数のスパンが必要な場合があるが、その場合はセミコロン ; で区切られる。

In [ ]:
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')

display(train.shape)


display(train.head())

In [ ]:
display(train.info())

**patient_Notes.csv** - 各患者のメモの説明はpatient_Notes.csvに、その説明から主な特徴はtrain.csvに書かれている。メモが与えられたとき、そのテキストの主な特徴または主なキーフレーズの位置を見つけるNERアプローチが必要で、場所は [start_character_index_of_main_feat_in_the_text:ending_character_index_of_main_feat_in_the_text] として格納されています。

**patient_Notes Description** -約40,000のPatient Noteの履歴部分を集めたもの。これらのうち、特徴量が注釈されているのはサブセットのみである。アノテーションのないノートに対して教師なし学習技術を適用したい場合がある。テストセット内の患者ノートは、このファイルの公開バージョンには含まれていません。

* pn_num** - 各患者ノートの一意な識別子。
* case_num** - 患者ノートが表す臨床ケースの一意な識別子です。
* pn_history** - テスト受験者によって記録された患者との出会いのテキスト。

In [ ]:
notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

display(notes.shape)

display(notes.head())

In [ ]:
notes['pn_history'][16][696:724]

In [ ]:
notes['pn_history'][16]

In [ ]:
notes.info()

**features.csv** - 各臨床例の特徴（またはキーコンセプト）のルーブリックです。

* **feature_num** - 各フィーチャーに一意の識別子。
* **case_num** - 各症例の一意の識別子。
* **feature_text** - 機能の説明。

In [ ]:
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')

display(features.shape)
features.head()

In [ ]:
features.info()

# EDA

**患者のノート**

In [ ]:
notes_count = notes.groupby('case_num').count()

fig = px.bar(data_frame=notes_count , x = notes_count.index , y='pn_num' , color='pn_num' , color_continuous_scale="Emrld"
             , title = 'Patient Notes per case')

fig.show()

In [ ]:
notes['word_count'] = notes['pn_history'].apply(lambda x : len(x.split()))

fig = px.histogram(notes , x ='word_count' , title='Patient History Word Count')

fig.show()

**特徴量**

In [ ]:
features_count = features.groupby('case_num').count()

fig = px.bar(data_frame=features_count , x = features_count.index , y='feature_num' , color='feature_num' , color_continuous_scale="Emrld"
             , title = 'Features Distribution per case')

fig.show()

In [ ]:
features['word_count'] = features['feature_text'].apply(lambda x : len(x.split('-')))

fig = px.histogram(features , x ='word_count' , title='Features Word Count')
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
train.head()

**患者の分析**

In [ ]:
print('Unique Patient Counts = ' , len(train['pn_num'].value_counts()))

In [ ]:
print('Data Frame for a particular patient')
train[train['pn_num'] == 16]

**アノテーション分析**

In [ ]:
print('Total Annotations = ', len(train['location']))
print('Number of Empty annotations and locations = ' , len(train[train['location'] == '[]']))

In [ ]:
train["location"] = train["location"].apply(eval) 
train['annotation'] = train['annotation'].apply(eval) # else whole list len will be counted as 1

train["annot_count"] = 0
for i in range(len(train)):
    train["annot_count"][i] = len(train["annotation"][i])
    
print('Annotation counts:')
print(train['annot_count'].value_counts().sort_index())

In [ ]:
fig = px.bar(data_frame=train , x = train['annot_count'].value_counts().sort_index().index ,
             y = train['annot_count'].value_counts().sort_index() , color = train['annot_count'].value_counts().sort_index() ,
             color_continuous_scale='Emrld' , title='Number of Annotations per row')

fig.update_xaxes(title= 'Number of Annotations')
fig.update_yaxes(title= 'Annotations Count')

fig.show()

In [ ]:
import spacy

patient_df = train[train["pn_num"] == 16]
location  = patient_df["location"]
annotation = patient_df["annotation"]
start_pos = []
end_pos = []
for i in location:
    for j in i:
        start_pos.append(j.split()[0])
        end_pos.append(j.split()[1])
        
ents = []
for i in range(len(start_pos)):
    ents.append({
        'start': int(start_pos[i]), 
        'end' : int(end_pos[i]),
        "label" : "Annotation"
    })
doc = {
    'text' : notes[notes["pn_num"] == 16]["pn_history"].iloc[0],
    "ents" : ents
}
colors = {"Annotation" :"linear-gradient(90deg, #aa9cfc, #fc9ce7)" } 
options = {"colors": colors}
spacy.displacy.render(doc, style="ent", options = options , manual=True, jupyter=True);

# ワードクラウド

**特徴量**

In [ ]:
concat_features = ' '.join([i for i in features.feature_text.astype(str)])
print(concat_features[:1000])

In [ ]:

stylecloud.gen_stylecloud(text=concat_features, icon_name= "fab fa-twitter", 
                          palette="cartocolors.diverging.TealRose_7", background_color="black" , size=1024)


In [ ]:
Image(filename="./stylecloud.png", width=1024, height=1024)

In [ ]:
features.feature_text.value_counts()[:30]

**患者の歴史**

In [ ]:
concat_pnhist = ''.join([i for i in notes.pn_history.astype(str)])

concat_pnhist[:200]

In [ ]:

stylecloud.gen_stylecloud(text=concat_pnhist, icon_name= "fas fa-dharmachakra", 
                          palette="cartocolors.diverging.TealRose_7", background_color="black" , size=1024)


In [ ]:
Image(filename="./stylecloud.png", width=1024, height=1024)

**アノテーション**

In [ ]:
concat_annot = ' '.join([i for i in train.annotation.astype(str)])
print(concat_annot[:500])

In [ ]:
stylecloud.gen_stylecloud(text=concat_annot,
                          icon_name='fas fa-yin-yang',
                          palette='colorbrewer.sequential.BuGn_9',
                          background_color='black',
                          gradient='horizontal',
                          size=1024)

In [ ]:
Image(filename="./stylecloud.png", width=1024, height=1024)